# Calculate the slope for each glacier

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/colili/Documents/PhD/project_john/DataFrame_produced_clean/df_center.csv', index_col=0)
df

/home/colili/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,rgiid,x,y,Surge,BgnDate,EndDate,CenLon,CenLat,Area,Zmin,...,check_geom,geometry,bed,thickness,surface_elevation,width_centerline,Runoff,ELA,beta,cmb
0,RGI60-07.00001,552858.197838,8.514877e+06,0,20080901,-9999999,17.03470,76.7052,0.597,142,...,NaN,NaN,401.83295,4.16705,406.0,86.307950,1.939792,489.989929,0.004318,-0.982753
1,RGI60-07.00001,552837.209663,8.514876e+06,0,20080901,-9999999,17.03470,76.7052,0.597,142,...,NaN,NaN,401.83295,4.16705,406.0,52.001931,1.940386,489.989929,0.004320,-0.983169
2,RGI60-07.00001,552816.221488,8.514875e+06,0,20080901,-9999999,17.03470,76.7052,0.597,142,...,NaN,NaN,401.83295,4.16705,406.0,63.412109,1.940980,489.989929,0.004321,-0.983585
3,RGI60-07.00001,552795.233313,8.514875e+06,0,20080901,-9999999,17.03470,76.7052,0.597,142,...,NaN,NaN,401.83295,4.16705,406.0,74.822291,1.941574,489.989929,0.004323,-0.984002
4,RGI60-07.00001,552774.245139,8.514874e+06,0,20080901,-9999999,17.03470,76.7052,0.597,142,...,NaN,NaN,401.83295,4.16705,406.0,112.223246,1.942168,489.989929,0.004324,-0.984418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134800,RGI60-07.01615,-329067.730001,8.041440e+06,9,19759999,-9999999,-8.34491,71.0142,0.098,351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134801,RGI60-07.01615,-329075.466520,8.041459e+06,9,19759999,-9999999,-8.34491,71.0142,0.098,351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134802,RGI60-07.01615,-329083.203033,8.041477e+06,9,19759999,-9999999,-8.34491,71.0142,0.098,351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134803,RGI60-07.01615,-329090.939541,8.041495e+06,9,19759999,-9999999,-8.34491,71.0142,0.098,351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calculate the slope per windows

Defining the window lenght and so the bed/surf slope of the window

Test on 1 glacier

In [3]:
data = df[df.rgiid=='RGI60-07.00004']
# gb = data.groupby(data.index // (data.shape[0]/10)).mean()
gb = data.groupby(data.index // (data.shape[0]/10))
start = gb.head(1)
end = gb.tail(1)
thick = abs(start.thickness.values - end.thickness.values)

# window_length = np.sqrt((start.x.values[0] - end.x.values[0])**2 + (start.y.values[0] - end.y.values[0])**2)
window_length = np.sqrt((start.x.values - end.x.values)**2 + (start.y.values - end.y.values)**2)
slope = np.arctan(thick/window_length)
data.columns

Index(['rgiid', 'x', 'y', 'Surge', 'BgnDate', 'EndDate', 'CenLon', 'CenLat',
       'Area', 'Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 'Status',
       'Connect', 'Form', 'TermType', 'Linkages', 'Name', 'check_geom',
       'geometry', 'bed', 'thickness', 'surface_elevation', 'width_centerline',
       'Runoff', 'ELA', 'beta', 'cmb'],
      dtype='object')

### Defining a function to apply it to all glaciers

In [4]:
def get_slope(data, n_windows=10):
    gb = data.groupby(data.index // (data.shape[0]/n_windows))
    start = gb.head(1)
    end = gb.tail(1)
#     thick = abs(start.thickness.values - end.thickness.values)
    thick_mean = (start.thickness.values + end.thickness.values)/2
    
#     bed = end.bed.values - start.bed.values
    bed = start.bed.values - end.bed.values
    surf = start.surface_elevation.values - end.surface_elevation.values
    
    window_length = np.sqrt((start.x.values - end.x.values)**2 + (start.y.values - end.y.values)**2)
#     if window_length.min() == 0:
#         print(start.x.values, end.x.values, start.y.values, end.y.values, window_length)
#         print(start.rgiid.unique())
#     bed_slope = np.arctan(thick/window_length)
    bed_slope = np.arctan(bed/window_length)
    surf_slope = np.arctan(surf/window_length)
    
    print(data.rgiid.unique()[0], surf_slope.shape, surf_slope.shape)
    
    return {
        'window_length':window_length
        , 'bed_slope':bed_slope
        , 'thick_mean':thick_mean
        , 'surf_slope':surf_slope
        , 'window_id':np.arange(0, surf_slope.shape[0], 1)
    }

get_slope(df[df.rgiid=='RGI60-07.00006'])

RGI60-07.00006 (11,) (11,)


{'window_length': array([484.24372617, 794.00057072, 647.30949668, 623.10231188,
        558.00895318, 647.30932742, 623.10214911, 723.04928907,
        834.13727515, 756.81742316, 186.00264518]),
 'bed_slope': array([        nan,  0.13633071,  0.05320125, -0.04913989,  0.05349393,
         0.06263766,  0.08490045,  0.05051171,  0.06327205,  0.0805607 ,
         0.21213162]),
 'thick_mean': array([       nan, 53.8075325, 96.708795 , 79.121972 , 48.8311405,
        44.569971 , 46.460073 , 55.0480985, 52.7419615, 47.717014 ,
        52.612419 ]),
 'surf_slope': array([       nan, 0.08668393, 0.02471266, 0.03689533, 0.06085569,
        0.06940687, 0.07369049, 0.04422815, 0.06942118, 0.08698722,
        0.17037273]),
 'window_id': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])}

### Get the number of window per glaciers

In [5]:
sloped = df.groupby('rgiid').apply(get_slope)

RGI60-07.00001 (10,) (10,)
RGI60-07.00002 (11,) (11,)
RGI60-07.00003 (11,) (11,)
RGI60-07.00004 (10,) (10,)
RGI60-07.00005 (11,) (11,)
RGI60-07.00006 (11,) (11,)
RGI60-07.00007 (11,) (11,)
RGI60-07.00008 (11,) (11,)
RGI60-07.00009 (11,) (11,)
RGI60-07.00010 (11,) (11,)
RGI60-07.00011 (10,) (10,)
RGI60-07.00012 (10,) (10,)
RGI60-07.00013 (10,) (10,)
RGI60-07.00014 (11,) (11,)
RGI60-07.00015 (11,) (11,)
RGI60-07.00016 (11,) (11,)
RGI60-07.00017 (11,) (11,)
RGI60-07.00018 (11,) (11,)
RGI60-07.00019 (11,) (11,)
RGI60-07.00020 (11,) (11,)
RGI60-07.00021 (11,) (11,)
RGI60-07.00022 (10,) (10,)
RGI60-07.00023 (11,) (11,)
RGI60-07.00024 (11,) (11,)
RGI60-07.00025 (11,) (11,)
RGI60-07.00026 (10,) (10,)
RGI60-07.00027 (11,) (11,)
RGI60-07.00028 (11,) (11,)
RGI60-07.00029 (11,) (11,)
RGI60-07.00030 (11,) (11,)
RGI60-07.00031 (11,) (11,)
RGI60-07.00032 (11,) (11,)
RGI60-07.00033 (11,) (11,)
RGI60-07.00034 (11,) (11,)
RGI60-07.00035 (11,) (11,)
RGI60-07.00036 (11,) (11,)
RGI60-07.00037 (11,) (11,)
R

<ipython-input-4-653c5f5beabd>:17: RuntimeWarning: invalid value encountered in true_divide
  bed_slope = np.arctan(bed/window_length)
<ipython-input-4-653c5f5beabd>:18: RuntimeWarning: invalid value encountered in true_divide
  surf_slope = np.arctan(surf/window_length)


RGI60-07.00063 (11,) (11,)
RGI60-07.00064 (11,) (11,)
RGI60-07.00065 (11,) (11,)
RGI60-07.00066 (11,) (11,)
RGI60-07.00067 (11,) (11,)
RGI60-07.00068 (11,) (11,)
RGI60-07.00069 (11,) (11,)
RGI60-07.00070 (11,) (11,)
RGI60-07.00071 (10,) (10,)
RGI60-07.00072 (11,) (11,)
RGI60-07.00073 (11,) (11,)
RGI60-07.00074 (11,) (11,)
RGI60-07.00075 (11,) (11,)
RGI60-07.00076 (11,) (11,)
RGI60-07.00077 (11,) (11,)
RGI60-07.00078 (11,) (11,)
RGI60-07.00079 (11,) (11,)
RGI60-07.00080 (11,) (11,)
RGI60-07.00081 (11,) (11,)
RGI60-07.00082 (11,) (11,)
RGI60-07.00083 (11,) (11,)
RGI60-07.00084 (11,) (11,)
RGI60-07.00085 (11,) (11,)
RGI60-07.00086 (11,) (11,)
RGI60-07.00087 (11,) (11,)
RGI60-07.00088 (11,) (11,)
RGI60-07.00089 (11,) (11,)
RGI60-07.00090 (11,) (11,)
RGI60-07.00091 (11,) (11,)
RGI60-07.00092 (11,) (11,)
RGI60-07.00093 (11,) (11,)
RGI60-07.00094 (11,) (11,)
RGI60-07.00095 (11,) (11,)
RGI60-07.00096 (11,) (11,)
RGI60-07.00097 (11,) (11,)
RGI60-07.00098 (11,) (11,)
RGI60-07.00099 (11,) (11,)
R

### Create dataframe that contains the slope, window information 

In [6]:
slope_df = pd.DataFrame()

for s in sloped.index:
    sdf = pd.DataFrame(sloped[s])
    sdf['rgiid'] = (s,)*sdf.shape[0]
    slope_df = pd.concat([slope_df, sdf])
slope_df

,window_length,bed_slope,thick_mean,surf_slope,window_id,rgiid
0,126.004294,0.273995,6.374700,0.241232,0,RGI60-07.00001
1,107.083040,0.308836,8.664474,0.307443,1,RGI60-07.00001
2,126.004250,0.406978,8.903227,0.404880,2,RGI60-07.00001
3,107.083003,0.179923,9.298506,0.175605,3,RGI60-07.00001
4,178.196857,0.192273,11.382651,0.172241,4,RGI60-07.00001
...,...,...,...,...,...,...
6,59.896867,NaN,NaN,NaN,6,RGI60-07.01615
7,59.896956,NaN,NaN,NaN,7,RGI60-07.01615
8,79.862742,NaN,NaN,NaN,8,RGI60-07.01615
9,59.897131,NaN,NaN,NaN,9,RGI60-07.01615


In [7]:
 df_slope = df[['rgiid', 'Name', 'Surge', 'BgnDate', 'EndDate', 'CenLon', 'CenLat',
       'Area', 'Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax']].drop_duplicates()

In [8]:
df_an = slope_df.merge(df_slope)
# analysis_df.thick_mean.fillna(-999, inplace=True)
# analysis_df.slope.fillna(-999, inplace=True)
df_an.dropna(subset=['thick_mean', 'bed_slope'], inplace=True)
df_an.columns

Index(['window_length', 'bed_slope', 'thick_mean', 'surf_slope', 'window_id',
       'rgiid', 'Name', 'Surge', 'BgnDate', 'EndDate', 'CenLon', 'CenLat',
       'Area', 'Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax'],
      dtype='object')

In [9]:
th = df_an.pivot_table(index='rgiid', columns='window_id', values='thick_mean')
th.columns = ['thick_w'+str(t) for t in th.columns]

wl = df_an.pivot_table(index='rgiid', columns='window_id', values='window_length')
wl.columns = ['windowlength_w'+str(t) for t in wl.columns]

bs = df_an.pivot_table(index='rgiid', columns='window_id', values='bed_slope')
bs.columns = ['bedslope_w'+str(t) for t in bs.columns]

ss = df_an.pivot_table(index='rgiid', columns='window_id', values='surf_slope')
ss.columns = ['surfslope_w'+str(t) for t in ss.columns]

In [10]:
df_analysis = df_an[['rgiid', 'Name', 'Surge', 'BgnDate', 'EndDate', 'CenLon', 'CenLat', 'Area', 'Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax']].drop_duplicates().set_index('rgiid')

In [11]:
df_window = df_analysis.join(th).join(wl).join(bs).join(ss).copy()

In [12]:
df_window

,Name,Surge,BgnDate,EndDate,CenLon,CenLat,Area,Zmin,Zmax,Zmed,...,surfslope_w1,surfslope_w2,surfslope_w3,surfslope_w4,surfslope_w5,surfslope_w6,surfslope_w7,surfslope_w8,surfslope_w9,surfslope_w10
rgiid,,,,,,,,,,,,,,,,,,,,,
RGI60-07.00001,NaN,0,20080901,-9999999,17.0347,76.7052,0.597,142,459,268,...,0.307443,0.404880,0.175605,0.172241,0.270833,0.082630,NaN,NaN,NaN,NaN
RGI60-07.00002,Bungebreen,0,20080901,-9999999,16.1891,76.7872,0.581,234,582,343,...,0.141895,0.319250,0.368304,0.165147,0.274176,0.171470,0.269394,0.153667,0.169084,NaN
RGI60-07.00003,Samarinbreen East,0,20080901,-9999999,16.3915,76.9194,0.388,157,540,330,...,0.309830,0.296274,0.237531,0.130825,0.102853,0.446205,0.468008,0.381731,NaN,NaN
RGI60-07.00004,NaN,0,20080901,-9999999,15.9932,76.9423,1.044,665,933,799,...,0.034655,0.165147,0.000000,0.000000,0.000000,0.000000,0.046192,0.000000,NaN,NaN
RGI60-07.00005,Mendelejevbreen,0,20080901,-9999999,16.4966,76.9564,0.500,186,396,263,...,0.736909,0.521469,0.000000,0.000000,0.000000,0.244612,-0.039978,0.000000,0.305425,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RGI60-07.01563,NaN,0,20020713,-9999999,18.7207,79.1558,8.875,21,689,479,...,0.165930,0.113673,0.122586,0.081001,0.077934,0.092045,0.080642,0.127409,NaN,NaN
RGI60-07.01564,Oslobreen,3,20020713,-9999999,18.2367,79.0561,330.996,24,1649,756,...,0.138452,0.044599,-0.036375,0.066663,0.079749,0.023170,0.028387,0.032944,0.026998,NaN
RGI60-07.01565,NaN,0,20040711,-9999999,15.0586,77.9282,0.699,335,714,534,...,NaN,0.188532,0.259992,0.157129,0.284950,0.243296,0.167869,0.280732,0.061828,NaN


In [13]:
df_window.to_csv('/home/colili/Documents/PhD/project_john/DataFrame_produced_clean/df_window.csv')